# Best X finder - Construction

Lo que haremos es construir el modelo descrito en el artículo. Sin embargo, el objetivo final, es diseñar una función que encuentre la variable independiente, $X$, con mayor significancia estadística para la variable dependiente $Y$. Este algoritmo funciona independientemente de si se trata un agente libre o no.

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from matplotlib.colors import ListedColormap
from termcolor import colored
from sklearn.naive_bayes import GaussianNB
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns
import pandas as pd
import numpy as np
import array
import math
import os
import warnings
print('Modulos importados')

Modulos importados


In [2]:
# Configuraciones
warnings.filterwarnings('ignore')
# Reduzcamos el número de línea a leer
pd.options.display.max_rows = 15

In [3]:
# Veamos el directorio actual de trabajo
print(os.getcwd())
# El directorio anterior es el correcto, pero si no lo fuese, hacemos lo sigueinte:
path = '/home/usuario/Documentos/Github/Proyectos/MLB_HN'
os.chdir(path)

/home/usuario/Documentos/Github/Proyectos/MLB_HN/Models/Linear_models/Free_agent/Best_X_finder


In [4]:
print("Para modificar el tamaño de todos los gráficos")
from matplotlib import rcParams
rcParams['figure.figsize'] = 15,9

Para modificar el tamaño de todos los gráficos


In [5]:
# Veamos el directorio actual de trabajo
print(os.getcwd())
# El directorio anterior es el correcto, pero si no lo fuese, hacemos lo sigueinte:
path = '/home/usuario/Documentos/Github/Proyectos/MLB_HN'
print("Nuevo directorio de trabajo: " + str(os.chdir(path)))

/home/usuario/Documentos/Github/Proyectos/MLB_HN
Nuevo directorio de trabajo: None


## Exploración de datos

Estudiaremos de manera general si hay datos faltantes, así como el tipo de datos que contiene

In [6]:
# Años de análisis
starting_year = 2011
period = 11
csv = '.csv'
# Directorios
pitchers_path = 'ETL_data/Free_Agent/Pitchers/free_agents_pitchers_'
hitters_path = 'ETL_data/Free_Agent/Hitters/free_agents_batters_'
# Originales:
df_pitchers = [None]*period
df_pitchers_copy = [None]*period
df_hitters = [None]*period
df_hitters_copy = [None]*period

Leamos todos las bases de datos correspondientes

In [7]:
for year in range(0,period):
    df_pitchers[year] = pd.read_csv(pitchers_path + str(starting_year + year) + csv)
    df_pitchers_copy[year] = df_pitchers[year].copy()
    
    df_hitters[year] = pd.read_csv(hitters_path + str(starting_year + year) + csv)
    df_hitters_copy[year] = df_hitters[year].copy()

Observemos el contenido de las bases de datos

In [8]:
# Pitchers
df_pitchers_copy[5].head()

,Acronimo,Altura,Anio,Anios_contrato,Bateos_pitcher,Bateos_pitcher_2,Cantidad de equipos,Cantidad_agentes_libres,Carreras,Carreras_2,...,Victorias,WAR,WAR_2,WHIP,WHIP_2,WS ganadas,Walks,Walks_2,Wins,Wins_2
0,TEX,6.5,2016,1,116,13456,2,10,68,4624,...,95,0.93,0.8649,1.36,1.8496,0,46,2116,7,49
1,CIN,6.6,2016,1,89,7921,2,8,64,4096,...,68,-2.14,4.5796,2.05,4.2025,5,31,961,2,4
2,DET,6.0,2013,5,171,29241,1,11,108,11664,...,86,-0.96,0.9216,1.46,2.1316,4,53,2809,7,49
3,MIL,6.3,2016,1,80,6400,1,8,30,900,...,72,0.60,0.3600,1.47,2.1609,0,17,289,2,4
4,CIN,6.5,2016,1,72,5184,2,8,38,1444,...,68,0.53,0.2809,1.44,2.0736,5,38,1444,6,36


In [9]:
list(enumerate(df_pitchers_copy[5].columns))

[(0, 'Acronimo'),
 (1, 'Altura'),
 (2, 'Anio'),
 (3, 'Anios_contrato'),
 (4, 'Bateos_pitcher'),
 (5, 'Bateos_pitcher_2'),
 (6, 'Cantidad de equipos'),
 (7, 'Cantidad_agentes_libres'),
 (8, 'Carreras'),
 (9, 'Carreras_2'),
 (10, 'Carreras_ganadas'),
 (11, 'Carreras_ganadas_2'),
 (12, 'Comando'),
 (13, 'Comando_2'),
 (14, 'Control'),
 (15, 'Control_2'),
 (16, 'Dominio'),
 (17, 'Dominio_2'),
 (18, 'ERA'),
 (19, 'ERA_2'),
 (20, 'Edad'),
 (21, 'Equipo'),
 (22, 'Equipo_anterior'),
 (23, 'Estado'),
 (24, 'Inning_pitched'),
 (25, 'Inning_pitched_2'),
 (26, 'Juegos'),
 (27, 'Juegos totales'),
 (28, 'Juegos_2'),
 (29, 'Juegos_iniciados'),
 (30, 'Juegos_iniciados_2'),
 (31, 'Jugador'),
 (32, 'Losses'),
 (33, 'Losses_2'),
 (34, 'Pennants won'),
 (35, 'Peso'),
 (36, 'Playoffs'),
 (37, 'Posicion'),
 (38, 'Promedio_victorias'),
 (39, 'Saves'),
 (40, 'Saves_2'),
 (41, 'Status'),
 (42, 'Strike-outs'),
 (43, 'Strike-outs_2'),
 (44, 'Sueldo'),
 (45, 'Sueldo_base'),
 (46, 'Sueldo_regular'),
 (47, 'TVS'),


In [10]:
# Hitters
df_hitters_copy[5].head()

,Acronimo,Altura,Anio,Anios_contrato,At-bats,At-bats_2,Bateos,Bateos_2,Cantidad de equipos,Cantidad_agentes_libres,...,Sueldo_regular,TVS,TVS_2,Valor_contrato,Valor_contrato_total,Valor_promedio_contrato,Victorias,WAR,WAR_2,WS ganadas
0,TEX,6.5,2016,1,2,4,0,0,2,10,...,493633,0.00,0.0000,507500,20165000,507500,95,0.93,0.8649,0
1,ATL,6.3,2016,1,247,61009,55,3025,1,13,...,3050000,0.00,0.0000,3000000,19157500,3000000,67,-1.28,1.6384,3
2,LAA,6.3,2012,10,593,351649,159,25281,5,9,...,25000000,20.37,414.9369,240000000,12525000,24000000,73,1.29,1.6641,1
3,NYM,6.0,2016,1,234,54756,48,2304,2,9,...,5750000,4.25,18.0625,5750000,48757500,5750000,87,-0.38,0.1444,2
4,KC,6.1,2016,4,445,198025,98,9604,2,10,...,12000000,12.36,152.7696,72000000,186750000,18000000,81,0.91,0.8281,2


Veamos las columnas con sus índices

In [11]:
list(enumerate(df_hitters_copy[5].columns))

[(0, 'Acronimo'),
 (1, 'Altura'),
 (2, 'Anio'),
 (3, 'Anios_contrato'),
 (4, 'At-bats'),
 (5, 'At-bats_2'),
 (6, 'Bateos'),
 (7, 'Bateos_2'),
 (8, 'Cantidad de equipos'),
 (9, 'Cantidad_agentes_libres'),
 (10, 'Edad'),
 (11, 'Equipo'),
 (12, 'Equipo_anterior'),
 (13, 'Estado'),
 (14, 'Home-runs'),
 (15, 'Home-runs_2'),
 (16, 'Juegos'),
 (17, 'Juegos totales'),
 (18, 'Juegos_2'),
 (19, 'Jugador'),
 (20, 'OPS'),
 (21, 'OPS_2'),
 (22, 'Pennants won'),
 (23, 'Peso'),
 (24, 'Playoffs'),
 (25, 'Porcentaje_bateo'),
 (26, 'Porcentaje_bateo_2'),
 (27, 'Porcetnaje_juegos'),
 (28, 'Porcetnaje_juegos_2'),
 (29, 'Posicion'),
 (30, 'Promedio_victorias'),
 (31, 'RBI'),
 (32, 'RBI_2'),
 (33, 'Status'),
 (34, 'Sueldo'),
 (35, 'Sueldo_base'),
 (36, 'Sueldo_regular'),
 (37, 'TVS'),
 (38, 'TVS_2'),
 (39, 'Valor_contrato'),
 (40, 'Valor_contrato_total'),
 (41, 'Valor_promedio_contrato'),
 (42, 'Victorias'),
 (43, 'WAR'),
 (44, 'WAR_2'),
 (45, 'WS ganadas')]

Veamos si hay bases de datos con variables que distintas entre sí

In [12]:
k = 0
for i in range(0, period - 1):
    if set(df_pitchers_copy[i].columns) != set(df_pitchers_copy[i + 1].columns):
        k = k + 1
    if set(df_hitters_copy[i].columns) != set(df_hitters_copy[i + 1].columns):
        k = k + 1
print("Número de dataframes con datos distintos: ",colored(k, "cyan"))    

Número de dataframes con datos distintos:  0


### Construcción de la base de datos para el modelo

#### Base de datos de los equipos

Como no hay ninguna base de datos distintas, hagamos un algoritmo que construya un dataframe que solo contenga a las variables $X$ y $Y$. Partamos de ver los datos que contiene cualquiera de los dataframes para orientar el algoritmo a uno de búsqueda dado el nombre de la variable, esto se hará tanto para *pitchers* como para los *hitters*.

In [13]:
# Names of variables in both dataframes
pitchers_variables = df_pitchers_copy[1].columns
hitters_variables = df_hitters_copy[1].columns

Lo que haremos es inicializar dos variables para cada dataframe: Nombre de la variable a buscar ($X$ y $Y$) y un dataframe vacío.

In [14]:
X_raw_hitter = 'Home-runs'
X_raw_pitcher = 'Inning_pitched'
Y_raw = 'Sueldo'
jugador = 'Jugador'
YX_pitcher_raw = [None]*period
YX_hitter_raw = [None]*period

In [15]:
for year in range(0, period):
    YX_pitcher_raw[year] = df_pitchers_copy[year][[jugador, Y_raw, X_raw_pitcher]]
    
    YX_hitter_raw[year] = df_hitters_copy[year][[jugador, Y_raw, X_raw_hitter]]

Ajustemos los precios por la inflación y cambiemos los nombres para facilitar el proceso *ETL*

In [16]:
change_inflation_percentage = {2011:20.46, 2012:18.02, 2013:16.32, 2014:14.46, 2015:14.33, 2016:12.90, 2017:10.55, 2018:7.86, 2019:5.99, 2020:4.70, 2021:1}

In [17]:
for year in range(0, period):
    YX_hitter_raw[year][[Y_raw]] = YX_hitter_raw[year][[Y_raw]]*(1 + change_inflation_percentage[2011 + year]*0.01)
    YX_hitter_raw[year].rename(columns = {Y_raw: Y_raw + '_' + str(2011 + year)}, inplace = True)
    YX_hitter_raw[year].rename(columns = {X_raw_hitter: X_raw_hitter + '_' + str(2011 + year)}, inplace = True)

    YX_pitcher_raw[year][[Y_raw]] = YX_pitcher_raw[year][[Y_raw]]*(1 + change_inflation_percentage[2011 + year]*0.01)
    YX_pitcher_raw[year].rename(columns = {Y_raw: Y_raw + '_' + str(2011 + year)}, inplace = True)
    YX_pitcher_raw[year].rename(columns = {X_raw_pitcher: X_raw_pitcher + '_' + str(2011 + year)}, inplace = True)

Observemos los resultados

In [18]:
YX_pitcher_raw[1]

,Jugador,Sueldo_2012,Inning_pitched_2012
0,C.J. Wilson,1.245111e+07,202.3
1,Mark Buehrle,8.261400e+06,202.3
2,Jonathan Papelbon,1.298227e+07,70.0
3,Heath Bell,8.261400e+06,63.7
4,Joe Nathan,9.500610e+06,64.3
...,...,...,...
12,Jamey Wright,1.062180e+06,67.7
13,Zach Duke,1.062180e+06,13.7
14,Jared Burton,8.851500e+05,62.0
15,Brad Penny,5.901000e+05,28.0


In [19]:
YX_hitter_raw[1]

,Jugador,Sueldo_2012,Home-runs_2012
0,Albert Pujols,1.416240e+07,30
1,Jose Reyes,1.180200e+07,11
2,C.J. Wilson,1.245111e+07,0
3,Mark Buehrle,8.261400e+06,0
4,Aramis Ramirez,7.081200e+06,27
...,...,...,...
30,Xavier Nady,8.261400e+05,4
31,Brad Penny,5.901000e+05,0
32,Bobby Abreu,1.014817e+07,6
33,Bobby Abreu,5.664960e+05,6


### Filtrado de datos

Contruiremos dos bases de datos tipo panel con el objetivo de facilitar la limpieza de datos

In [20]:
YX_pitcher_panel_raw = pd.merge(YX_pitcher_raw[0], YX_pitcher_raw[1], how = 'outer', on = 'Jugador')

for year in range(2,period):
    YX_pitcher_panel_raw = pd.merge(YX_pitcher_panel_raw, YX_pitcher_raw[year], how = 'outer', on = 'Jugador')

In [21]:
YX_pitcher_panel_raw.drop_duplicates(subset = 'Jugador', inplace = True)
YX_pitcher_panel_raw = YX_pitcher_panel_raw.sort_values(by = 'Jugador', ascending = True)
YX_pitcher_panel_raw.reset_index(drop = True, inplace = True)
YX_pitcher_panel_raw.sort_index(axis = 1, inplace = True)
YX_pitcher_panel_raw.tail()

,Inning_pitched_2011,Inning_pitched_2012,Inning_pitched_2013,Inning_pitched_2014,Inning_pitched_2015,Inning_pitched_2016,Inning_pitched_2017,Inning_pitched_2018,Inning_pitched_2019,Inning_pitched_2020,...,Sueldo_2012,Sueldo_2013,Sueldo_2014,Sueldo_2015,Sueldo_2016,Sueldo_2017,Sueldo_2018,Sueldo_2019,Sueldo_2020,Sueldo_2021
327,NaN,13.7,31.3,58.7,61.7,60.8,18.3,NaN,NaN,NaN,...,1062180.0,569968.0,972910.0,5144850.0,5645000.0,6080250.0,NaN,NaN,NaN,NaN
328,NaN,NaN,NaN,54.7,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,572300.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
329,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61.3,19.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13778700.0,13611000.0,13130000.0
330,NaN,NaN,177.7,202.3,NaN,158.7,202.3,207.7,208.7,67.0,...,NaN,22100800.0,29759600.0,NaN,38386000.0,37587000.0,36672400.0,36566550.0,36645000.0,35350000.0
331,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,71.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22510500.0,NaN


Para facilitar los algoritmos, movamos la columna que contiene el nombre del jugador en la primera posición.

In [22]:
player_column_pitcher = YX_pitcher_panel_raw.pop('Jugador')
YX_pitcher_panel_raw.insert(0, 'Jugador', player_column_pitcher)
YX_pitcher_panel_raw.tail()

,Jugador,Inning_pitched_2011,Inning_pitched_2012,Inning_pitched_2013,Inning_pitched_2014,Inning_pitched_2015,Inning_pitched_2016,Inning_pitched_2017,Inning_pitched_2018,Inning_pitched_2019,...,Sueldo_2012,Sueldo_2013,Sueldo_2014,Sueldo_2015,Sueldo_2016,Sueldo_2017,Sueldo_2018,Sueldo_2019,Sueldo_2020,Sueldo_2021
327,Zach Duke,NaN,13.7,31.3,58.7,61.7,60.8,18.3,NaN,NaN,...,1062180.0,569968.0,972910.0,5144850.0,5645000.0,6080250.0,NaN,NaN,NaN,NaN
328,Zach Putnam,NaN,NaN,NaN,54.7,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,572300.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
329,Zack Britton,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13778700.0,13611000.0,13130000.0
330,Zack Greinke,NaN,NaN,177.7,202.3,NaN,158.7,202.3,207.7,208.7,...,NaN,22100800.0,29759600.0,NaN,38386000.0,37587000.0,36672400.0,36566550.0,36645000.0,35350000.0
331,Zack Wheeler,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22510500.0,NaN


In [23]:
YX_pitcher_panel_raw.columns

Index(['Jugador', 'Inning_pitched_2011', 'Inning_pitched_2012',
       'Inning_pitched_2013', 'Inning_pitched_2014', 'Inning_pitched_2015',
       'Inning_pitched_2016', 'Inning_pitched_2017', 'Inning_pitched_2018',
       'Inning_pitched_2019', 'Inning_pitched_2020', 'Inning_pitched_2021',
       'Sueldo_2011', 'Sueldo_2012', 'Sueldo_2013', 'Sueldo_2014',
       'Sueldo_2015', 'Sueldo_2016', 'Sueldo_2017', 'Sueldo_2018',
       'Sueldo_2019', 'Sueldo_2020', 'Sueldo_2021'],
      dtype='object')

In [24]:
print(YX_pitcher_panel_raw.shape)

(332, 23)


Lo que ahora queremos es dividir el dataframe en la variable dependiente e independiente, para ello crearemos un número guía que guarde el índice de la columna a partir de la cual se hará la división. Este índice de guía será cierto para todo tipo de variables mientras el periodo de análisis sea el mismo

In [25]:
split_column_pitcher = 1 + period
print("Nombres ordenados: ", "\n", colored(YX_pitcher_panel_raw.columns, "green"), "\n")
print("Con índice: ", "\n", colored(list(enumerate(YX_pitcher_panel_raw.columns)), "green"), "\n")
print("Índice de columna guía: ", colored(split_column_pitcher, "blue"), '\n')

Nombres ordenados:  
 Index(['Jugador', 'Inning_pitched_2011', 'Inning_pitched_2012',
       'Inning_pitched_2013', 'Inning_pitched_2014', 'Inning_pitched_2015',
       'Inning_pitched_2016', 'Inning_pitched_2017', 'Inning_pitched_2018',
       'Inning_pitched_2019', 'Inning_pitched_2020', 'Inning_pitched_2021',
       'Sueldo_2011', 'Sueldo_2012', 'Sueldo_2013', 'Sueldo_2014',
       'Sueldo_2015', 'Sueldo_2016', 'Sueldo_2017', 'Sueldo_2018',
       'Sueldo_2019', 'Sueldo_2020', 'Sueldo_2021'],
      dtype='object') 

Con índice:  
 [(0, 'Jugador'), (1, 'Inning_pitched_2011'), (2, 'Inning_pitched_2012'), (3, 'Inning_pitched_2013'), (4, 'Inning_pitched_2014'), (5, 'Inning_pitched_2015'), (6, 'Inning_pitched_2016'), (7, 'Inning_pitched_2017'), (8, 'Inning_pitched_2018'), (9, 'Inning_pitched_2019'), (10, 'Inning_pitched_2020'), (11, 'Inning_pitched_2021'), (12, 'Sueldo_2011'), (13, 'Sueldo_2012'), (14, 'Sueldo_2013'), (15, 'Sueldo_2014'), (16, 'Sueldo_2015'), (17, 'Sueldo_2016'), (18, 'S

Esta columna guía es a partir donde iniciará la otra variable.

Guardaremos los índices de los jugadores que contengan menos de dos datos

In [26]:
max_nan = 2
print("Máximo número de 'nan' que pueden haber en un renglón: ", colored(max_nan, "red"))

Máximo número de 'nan' que pueden haber en un renglón:  2


In [27]:
pitchers_ini_len = YX_pitcher_panel_raw.shape[0]
print("Cantidad de renglones iniciales para la variable independiente: ", colored(pitchers_ini_len, "green"))

Cantidad de renglones iniciales para la variable independiente:  332


Dependiendo del orden alfabético relativo entre **X_raw_hitter** y **Y_raw**.

In [28]:
if Y_raw > X_raw_pitcher:
    X_pitcher_drop_etl = YX_pitcher_panel_raw.iloc[:, 1:split_column_pitcher]
else:
    X_pitcher_drop_etl = YX_pitcher_panel_raw.iloc[:, split_column_pitcher:YX_pitcher_panel_raw.shape[1]]

In [29]:
X_pitcher_drop_etl.head()

,Inning_pitched_2011,Inning_pitched_2012,Inning_pitched_2013,Inning_pitched_2014,Inning_pitched_2015,Inning_pitched_2016,Inning_pitched_2017,Inning_pitched_2018,Inning_pitched_2019,Inning_pitched_2020,Inning_pitched_2021
0,NaN,NaN,NaN,213.7,164.0,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,119.0,NaN,NaN,NaN,NaN,NaN
2,NaN,179.7,143.3,NaN,172.3,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66.3,18.3,80.3


In [30]:
nulls_quantity = period - max_nan
X_pitcher_drop_etl = X_pitcher_drop_etl.iloc[X_pitcher_drop_etl[(X_pitcher_drop_etl.isnull().sum(axis = 1) > nulls_quantity)].index]

In [31]:
X_pitcher_drop_etl.head()

,Inning_pitched_2011,Inning_pitched_2012,Inning_pitched_2013,Inning_pitched_2014,Inning_pitched_2015,Inning_pitched_2016,Inning_pitched_2017,Inning_pitched_2018,Inning_pitched_2019,Inning_pitched_2020,Inning_pitched_2021
1,NaN,NaN,NaN,NaN,NaN,119.0,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.7,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,56.0,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,87.3


In [32]:
print("Cantidad de jugadores que no cumplen la condición para la variable independiente: \n", colored(X_pitcher_drop_etl.shape[0], "red"))

Cantidad de jugadores que no cumplen la condición para la variable independiente: 
 153


Crearemos un *array* para guardar los índices de los renglones que estén en el dataframe *X_drop_etl*. Son los que no cumplen la condición mencionada anteriormente

In [33]:
X_pitcher_index_drop = X_pitcher_drop_etl.index

Repetiremos el mismo proceso para la variable del salario ajustado de los pitcher

In [34]:
if Y_raw > X_raw_pitcher:
    Y_pitcher_drop_etl = YX_pitcher_panel_raw.iloc[:, split_column_pitcher:YX_pitcher_panel_raw.shape[1]]
else:
    Y_pitcher_drop_etl = YX_pitcher_panel_raw.iloc[:, 1:split_column_pitcher]

In [35]:
Y_pitcher_drop_etl.head()

,Sueldo_2011,Sueldo_2012,Sueldo_2013,Sueldo_2014,Sueldo_2015,Sueldo_2016,Sueldo_2017,Sueldo_2018,Sueldo_2019,Sueldo_2020,Sueldo_2021
0,NaN,NaN,NaN,12876750.0,9718050.0,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,572967.5,NaN,NaN,NaN,NaN,NaN
2,NaN,3540600.0,6397600.0,NaN,5716500.0,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,13719600.0,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9539100.0,9423000.0,7297294.44


In [36]:
Y_pitcher_drop_etl = Y_pitcher_drop_etl.iloc[Y_pitcher_drop_etl[(Y_pitcher_drop_etl.isnull().sum(axis = 1) > nulls_quantity)].index]
Y_pitcher_index_drop = Y_pitcher_drop_etl.index

Luego, eleminemos esos índices del dataframe del panel data

In [37]:
pitcher_index_drop = list(set(Y_pitcher_index_drop).union(set(X_pitcher_index_drop)))

In [38]:
print("Cantidad de jugadores que se eliminarán: \n", colored(len(pitcher_index_drop), "red"))

Cantidad de jugadores que se eliminarán: 
 153


In [39]:
YX_pitcher_panel_raw.drop(index = pitcher_index_drop, inplace = True)
YX_pitcher_panel_raw.reset_index(drop = True, inplace = True)
YX_pitcher_panel_raw = YX_pitcher_panel_raw.sort_values(by = 'Jugador', ascending = True)
print("Cantidad de jugadores restantes en total: \n", colored(YX_pitcher_panel_raw.shape, "cyan"))

Cantidad de jugadores restantes en total: 
 (179, 23)


In [40]:
YX_pitcher_panel_raw.head()

,Jugador,Inning_pitched_2011,Inning_pitched_2012,Inning_pitched_2013,Inning_pitched_2014,Inning_pitched_2015,Inning_pitched_2016,Inning_pitched_2017,Inning_pitched_2018,Inning_pitched_2019,...,Sueldo_2012,Sueldo_2013,Sueldo_2014,Sueldo_2015,Sueldo_2016,Sueldo_2017,Sueldo_2018,Sueldo_2019,Sueldo_2020,Sueldo_2021
0,A.J. Burnett,NaN,NaN,NaN,213.7,164.0,NaN,NaN,NaN,NaN,...,NaN,NaN,12876750.0,9718050.0,NaN,NaN,NaN,NaN,NaN,NaN
1,Aaron Harang,NaN,179.7,143.3,NaN,172.3,NaN,NaN,NaN,NaN,...,3540600.0,6397600.0,NaN,5716500.0,NaN,NaN,NaN,NaN,NaN,NaN
2,Adam Ottavino,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9539100.0,9.423000e+06,7297294.44
3,Adam Wainwright,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.398333e+06,8080000.00
4,Alex Cobb,NaN,NaN,NaN,NaN,NaN,NaN,NaN,152.3,12.3,...,NaN,NaN,NaN,NaN,NaN,NaN,15100400.0,14838600.0,1.465800e+07,15150000.00


In [41]:
print("Tamaño: \n", colored(YX_pitcher_panel_raw.shape, "cyan"))

Tamaño: 
 (179, 23)


Debido a que pueden haber estadísticas deportivas que sean iguales a cero, es necesario tratar dicho datos. En el caso de las estadísticas analizadas, no hay valores negativos, por lo que no tenemos que tratar dichos datos. **POR EL MOMENTO SOLO SE ELIMINARÁN DICHOS RENGLONES, PERO DESPUÉS SE HARÁ UNA TRANSFORMACIÓN PARA NO PERDER ESOS DATOS.** Creemos una columna que alamcene los máximos por renglón de la estadística deportiva para después filtrar los renglones cuyo máximo sea $0$.

In [42]:
if Y_raw > X_raw_pitcher:
    YX_pitcher_panel_raw[X_raw_pitcher + '_max'] = YX_pitcher_panel_raw.iloc[:, 1:split_column_pitcher].max(axis = 1)
else:
    YX_pitcher_panel_raw[X_raw_pitcher + '_max'] = YX_pitcher_panel_raw.iloc[:, split_column_pitcher:YX_pitcher_panel_raw.shape[1]].max(axis = 1)

In [43]:
YX_pitcher_max_0 = YX_pitcher_panel_raw[YX_pitcher_panel_raw[X_raw_pitcher + '_max'] == 0]

In [44]:
YX_pitcher_max_0.head()

,Jugador,Inning_pitched_2011,Inning_pitched_2012,Inning_pitched_2013,Inning_pitched_2014,Inning_pitched_2015,Inning_pitched_2016,Inning_pitched_2017,Inning_pitched_2018,Inning_pitched_2019,...,Sueldo_2013,Sueldo_2014,Sueldo_2015,Sueldo_2016,Sueldo_2017,Sueldo_2018,Sueldo_2019,Sueldo_2020,Sueldo_2021,Inning_pitched_max


Independientemente de que haya o no filas que cumplan esta condición, podremos ejecutar la intrucción para borrar dichas filas

In [45]:
YX_pitcher_panel_raw.drop(columns = X_raw_pitcher + '_max', inplace = True)

In [46]:
YX_pitcher_max_0_index_drop = YX_pitcher_max_0.index

In [47]:
YX_pitcher_panel_raw.drop(index = YX_pitcher_max_0_index_drop, inplace = True)
YX_pitcher_panel_raw.reset_index(drop = True, inplace = True)
print("Cantidad de jugadores restantes en total: \n", colored(YX_pitcher_panel_raw.shape, "cyan"))

Cantidad de jugadores restantes en total: 
 (179, 23)


In [48]:
YX_pitcher_panel_raw.head()

,Jugador,Inning_pitched_2011,Inning_pitched_2012,Inning_pitched_2013,Inning_pitched_2014,Inning_pitched_2015,Inning_pitched_2016,Inning_pitched_2017,Inning_pitched_2018,Inning_pitched_2019,...,Sueldo_2012,Sueldo_2013,Sueldo_2014,Sueldo_2015,Sueldo_2016,Sueldo_2017,Sueldo_2018,Sueldo_2019,Sueldo_2020,Sueldo_2021
0,A.J. Burnett,NaN,NaN,NaN,213.7,164.0,NaN,NaN,NaN,NaN,...,NaN,NaN,12876750.0,9718050.0,NaN,NaN,NaN,NaN,NaN,NaN
1,Aaron Harang,NaN,179.7,143.3,NaN,172.3,NaN,NaN,NaN,NaN,...,3540600.0,6397600.0,NaN,5716500.0,NaN,NaN,NaN,NaN,NaN,NaN
2,Adam Ottavino,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9539100.0,9.423000e+06,7297294.44
3,Adam Wainwright,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.398333e+06,8080000.00
4,Alex Cobb,NaN,NaN,NaN,NaN,NaN,NaN,NaN,152.3,12.3,...,NaN,NaN,NaN,NaN,NaN,NaN,15100400.0,14838600.0,1.465800e+07,15150000.00


Puesto que ya nos encargamos de toda la limpieza y tratamiento, ya podremos crear las variables $X$ y $Y$ para la regresión.

### Variable $X$

$$
X = \left( -1 \right)^{I_{t}^{-}}\frac{y_{t}}{\sqrt{y_H}}
$$

Obtengamos la variable $X$. Hallemos primero los máximos y mínimos por renglón.

In [49]:
if Y_raw > X_raw_pitcher:
    X_pitcher_etl = YX_pitcher_panel_raw.iloc[:, 1:split_column_pitcher]
else:
    X_pitcher_etl = YX_pitcher_panel_raw.iloc[:, split_column_pitcher:YX_pitcher_panel_raw.shape[1]]

In [50]:
pitcher_max_element = X_pitcher_etl.max(axis = 1)
pitcher_min_element = X_pitcher_etl.min(axis = 1)

In [51]:
pitcher_max_min = (pitcher_max_element + pitcher_min_element)/2

Generemos las listas que contengan los nombres de las nuevas variables: La dummy auxiliar $I$ y $X$.

In [52]:
dummy_pitcher_names = []
X_pitcher_names = []

for year in range(0,period):
    dummy_pitcher_names.append("I_" + str(2011 + year))
    X_pitcher_names.append("X_" + str(2011 + year))

In [53]:
print(dummy_pitcher_names)
print()
print(X_pitcher_names)

['I_2011', 'I_2012', 'I_2013', 'I_2014', 'I_2015', 'I_2016', 'I_2017', 'I_2018', 'I_2019', 'I_2020', 'I_2021']

['X_2011', 'X_2012', 'X_2013', 'X_2014', 'X_2015', 'X_2016', 'X_2017', 'X_2018', 'X_2019', 'X_2020', 'X_2021']


Ahora, creemos la variable dummy $I$

In [54]:
for year in range(0,period):
    X_pitcher_conditions = [
    (X_pitcher_etl.iloc[:,year] == pitcher_max_element) & ~(X_pitcher_etl.iloc[:,year].isnull()),
    (X_pitcher_etl.iloc[:,year] != pitcher_max_element) & ~(X_pitcher_etl.iloc[:,year].isnull())
    ]

    X_pitcher_conditions_values = [0,1]
    
    X_pitcher_etl[dummy_pitcher_names[year]] = np.select(X_pitcher_conditions, X_pitcher_conditions_values, default = np.nan)

In [55]:
X_pitcher_etl.columns

Index(['Inning_pitched_2011', 'Inning_pitched_2012', 'Inning_pitched_2013',
       'Inning_pitched_2014', 'Inning_pitched_2015', 'Inning_pitched_2016',
       'Inning_pitched_2017', 'Inning_pitched_2018', 'Inning_pitched_2019',
       'Inning_pitched_2020', 'Inning_pitched_2021', 'I_2011', 'I_2012',
       'I_2013', 'I_2014', 'I_2015', 'I_2016', 'I_2017', 'I_2018', 'I_2019',
       'I_2020', 'I_2021'],
      dtype='object')

Veamos las columnas de las dummies

In [56]:
X_pitcher_etl.iloc[:,period:]

,I_2011,I_2012,I_2013,I_2014,I_2015,I_2016,I_2017,I_2018,I_2019,I_2020,I_2021
0,NaN,NaN,NaN,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,0.0,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
174,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,0.0
175,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,1.0,NaN,0.0
176,NaN,1.0,1.0,1.0,0.0,1.0,1.0,NaN,NaN,NaN,NaN
177,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,1.0


Para facilitar los algoritmos, separaremos las variables sobre las estadísticas deportivas y dummies en dos dataframes distintos. Debido a cómo se construyeron estas variables en la base de datos, no es necesario preocuparnos por el orden alfabético relativo entre los nombres de las variables.

**Variables estadística:**

In [57]:
X_pitcher_aux = X_pitcher_etl.iloc[:,:period]
X_pitcher_aux.head()

,Inning_pitched_2011,Inning_pitched_2012,Inning_pitched_2013,Inning_pitched_2014,Inning_pitched_2015,Inning_pitched_2016,Inning_pitched_2017,Inning_pitched_2018,Inning_pitched_2019,Inning_pitched_2020,Inning_pitched_2021
0,NaN,NaN,NaN,213.7,164.0,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,179.7,143.3,NaN,172.3,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66.3,18.3,80.3
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65.7,272.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,152.3,12.3,52.3,145.7


**Variables dummie:**

In [58]:
Dummy_pitcher = X_pitcher_etl.iloc[:,period:]
Dummy_pitcher.head()

,I_2011,I_2012,I_2013,I_2014,I_2015,I_2016,I_2017,I_2018,I_2019,I_2020,I_2021
0,NaN,NaN,NaN,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,0.0,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,1.0,1.0


Usemos la variable dummy para crear finalmente a la variable $X$. Sin embargo, primero creemos una serie auxiliar que contenga el máximo de cada renglón

In [59]:
X_pitcher_max = X_pitcher_aux.max(axis = 1)
X_pitcher_aux[['Max']] = np.sqrt(X_pitcher_max)

In [60]:
for year in range(0,period):    
    X_pitcher_etl['X_' + str(2011 + year)] = \
    (X_pitcher_aux[X_raw_pitcher + '_' + str(2011 + year)]*
    ((-1)**Dummy_pitcher['I_' + str(2011 + year)]))

In [61]:
X_pitcher_etl = X_pitcher_etl.div(X_pitcher_aux['Max'].values, axis = 0)

Como ya no nos interesa la variable sobre la estadística deportiva en crudo, solo conservaremos las columnas de la variables $X$

In [62]:
X_pitcher_etl = X_pitcher_etl.iloc[:,2*period:]

In [63]:
X_pitcher_etl.head()

,X_2011,X_2012,X_2013,X_2014,X_2015,X_2016,X_2017,X_2018,X_2019,X_2020,X_2021
0,NaN,NaN,NaN,14.618481,-11.218676,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,13.405223,-10.689863,NaN,-12.853199,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-7.398706,-2.042177,8.961027
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-3.983648,16.492423
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.340989,-0.996679,-4.237910,-11.806185


Recorramos hasta la izquierda todos los valores del dataframe para facilitar los algoritmos

In [64]:
for i in range(0,X_pitcher_etl.shape[0]):
    row = X_pitcher_etl.iloc[i]
    row_length = row.size
    sustitute = []
    
    for j in range(row_length):
        if pd.isna(row[j]) != True:
            sustitute.append(row[j])
            
    sustitute_length = len(sustitute)
    
    for k in range(row_length - sustitute_length):
        sustitute.append(np.nan)
        
    for j in range(row_length):
        X_pitcher_etl.iloc[i,j] = sustitute[j]

X_pitcher_etl.columns = ['X_1', 'X_2', 'X_3', 'X_4', 'X_5', 'X_6', 
                         'X_7', 'X_8', 'X_9', 'X_10', 'X_11']

In [65]:
X_pitcher_etl.head()

,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10,X_11
0,14.618481,-11.218676,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,13.405223,-10.689863,-12.853199,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-7.398706,-2.042177,8.961027,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-3.983648,16.492423,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,12.340989,-0.996679,-4.237910,-11.806185,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Como tiene las mismas filas que el dataframe **YX_pitcher_panel_raw**, podemos añadir la columna de jugadores sin problemas

In [66]:
X_pitcher_etl[['Jugador']] = YX_pitcher_panel_raw[['Jugador']]
X_player_column_pitcher = X_pitcher_etl.pop('Jugador')
X_pitcher_etl.insert(0, 'Jugador', X_player_column_pitcher)
X_pitcher_etl.head()

,Jugador,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10,X_11
0,A.J. Burnett,14.618481,-11.218676,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Aaron Harang,13.405223,-10.689863,-12.853199,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Adam Ottavino,-7.398706,-2.042177,8.961027,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Adam Wainwright,-3.983648,16.492423,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Alex Cobb,12.340989,-0.996679,-4.237910,-11.806185,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Variable $Y$

$$
Y = \sqrt{\omega_{t+1}} - \sqrt{\omega_{t}}
$$

Usaremos de nuevo un dataframe auziliar

In [67]:
if Y_raw > X_raw_pitcher:
    Y_pitcher_etl = YX_pitcher_panel_raw.iloc[:, split_column_pitcher:YX_pitcher_panel_raw.shape[1]]
else:
    Y_pitcher_etl = YX_pitcher_panel_raw.iloc[:, 1:split_column_pitcher]

In [68]:
Y_pitcher_etl.head()

,Sueldo_2011,Sueldo_2012,Sueldo_2013,Sueldo_2014,Sueldo_2015,Sueldo_2016,Sueldo_2017,Sueldo_2018,Sueldo_2019,Sueldo_2020,Sueldo_2021
0,NaN,NaN,NaN,12876750.0,9718050.0,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,3540600.0,6397600.0,NaN,5716500.0,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9539100.0,9.423000e+06,7297294.44
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.398333e+06,8080000.00
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15100400.0,14838600.0,1.465800e+07,15150000.00


In [69]:
for i in range(0,Y_pitcher_etl.shape[0]):
    row = Y_pitcher_etl.iloc[i]
    row_length = row.size
    sustitute = []
    
    for j in range(row_length):
        if pd.isna(row[j]) != True:
            sustitute.append(row[j])
            
    sustitute_length = len(sustitute)
    
    for k in range(row_length - sustitute_length):
        sustitute.append(np.nan)
        
    for j in range(row_length):
        Y_pitcher_etl.iloc[i,j] = sustitute[j]

Y_pitcher_etl.columns = ['w_1', 'w_2', 'w_3', 'w_4', 'w_5', 'w_6', 
                         'w_7', 'w_8', 'w_9', 'w_10', 'w_11']

In [70]:
Y_pitcher_etl[['Jugador']] = YX_pitcher_panel_raw[['Jugador']]
Y_player_column_pitcher = Y_pitcher_etl.pop('Jugador')
Y_pitcher_etl.insert(0, 'Jugador', Y_player_column_pitcher)
Y_pitcher_etl.head()

,Jugador,w_1,w_2,w_3,w_4,w_5,w_6,w_7,w_8,w_9,w_10,w_11
0,A.J. Burnett,12876750.0,9718050.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Aaron Harang,3540600.0,6397600.0,5716500.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Adam Ottavino,9539100.0,9423000.0,7297294.44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Adam Wainwright,6398333.217,8080000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Alex Cobb,15100400.0,14838600.0,14658000.00,15150000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [71]:
Y_pitcher_etl_copy = Y_pitcher_etl.copy()
y_etl_names = Y_pitcher_etl_copy['Jugador']

Apliquemos el algoritmo para clacular a $Y$

In [72]:
for i in range(0,len(y_etl_names)):
    row = Y_pitcher_etl_copy.iloc[i]
    row_length = row.size
    k = 0

    sustitute = []

    for j in range(0, row_length):
        if pd.isna(row[j]) != True:
            k = k + 1

    sustitute.append(y_etl_names[i])
    for j in range(1,k - 1):
        sustitute.append(row[j + 1]**0.5 - row[j]**0.5)

    sustitute_length = len(sustitute)

    for k in range(row_length - sustitute_length):
        sustitute.append(np.nan)

    for j in range(row_length):
        Y_pitcher_etl_copy.iloc[i,j] = sustitute[j]

Y_pitcher_etl_copy.columns = ['Jugador', 'Y_1', 'Y_2', 'Y_3', 'Y_4', 'Y_5', 'Y_6', 
                              'Y_7', 'Y_8', 'Y_9', 'Y_10', 'Y_11']

In [73]:
Y_pitcher_etl = Y_pitcher_etl_copy
Y_pitcher_etl.head()

,Jugador,Y_1,Y_2,Y_3,Y_4,Y_5,Y_6,Y_7,Y_8,Y_9,Y_10,Y_11
0,A.J. Burnett,-471.040165,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Aaron Harang,647.699528,-138.427417,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Adam Ottavino,-18.852808,-368.340054,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Adam Wainwright,313.041401,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Alex Cobb,-33.832972,-23.513583,63.723332,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Verifiquemos que ambos dataframes contengan la misma cantidad de filas

In [74]:
print("Y_pitcher_etl: \n", colored(Y_pitcher_etl.shape, "cyan"))
print("X_pitcher_etl: \n", colored(X_pitcher_etl.shape, "cyan"))

Y_pitcher_etl: 
 (179, 12)
X_pitcher_etl: 
 (179, 12)


La estrategia para construir la base de datos para la regresión consiste en el siguiente proceso:
- Crear dataframes que contengan la columna del nombre del jugador y la variable de $X$ y $Y$ del mismo subíndice.
- Hacer merge de las bases de datos que contengan el mismo subíndice.
- Concatenar todas las bases de datos.
- Eliminar las filas que contengan datos faltantes.

In [75]:
regression_pitcher_list = [None]*(period-1)

In [76]:
for i in range(0,len(regression_pitcher_list)):
    merge = pd.merge(Y_pitcher_etl.iloc[:,[0,i+1]], X_pitcher_etl.iloc[:,[0,i+1]])
    regression_pitcher_list[i] = merge
    regression_pitcher_list[i].columns = ['Jugador', 'Y', 'X']

In [77]:
df_article_regression_pitcher = regression_pitcher_list[0]

for i in range(1,len(regression_pitcher_list)):
    df_article_regression_pitcher = pd.concat([df_article_regression_pitcher, regression_pitcher_list[i]])

In [78]:
df_article_regression_pitcher = df_article_regression_pitcher.sort_values(by = 'Jugador', ascending = True)
df_article_regression_pitcher.reset_index(drop = True, inplace = True)
df_article_regression_pitcher

,Jugador,Y,X
0,A.J. Burnett,-471.040165,14.618481
1,A.J. Burnett,NaN,NaN
2,A.J. Burnett,NaN,NaN
3,A.J. Burnett,NaN,NaN
4,A.J. Burnett,NaN,NaN
...,...,...,...
1785,Zack Greinke,-64.819869,-10.286993
1786,Zack Greinke,740.407614,-13.113161
1787,Zack Greinke,754.087221,-11.51858
1788,Zack Greinke,NaN,NaN


Por último, eliminemos las filas que contengan filas con datos *NaN*.

In [79]:
df_article_regression_pitcher.dropna(inplace = True)
df_article_regression_pitcher.reset_index(drop = True, inplace = True)
df_article_regression_pitcher

,Jugador,Y,X
0,A.J. Burnett,-471.040165,14.618481
1,Aaron Harang,647.699528,13.405223
2,Aaron Harang,-138.427417,-10.689863
3,Adam Ottavino,-368.340054,-2.042177
4,Adam Ottavino,-18.852808,-7.398706
...,...,...,...
420,Zack Greinke,6.483182,-13.528012
421,Zack Greinke,-75.049658,-13.113161
422,Zack Greinke,-64.819869,-10.286993
423,Zack Greinke,740.407614,-13.113161


In [80]:
print("Cantidad de jugadores para la regresión: \n", colored(df_article_regression_pitcher.shape, "cyan"))

Cantidad de jugadores para la regresión: 
 (425, 3)


## Replicación del algoritmo

Ahora, repetiremos en un solo bloque de código el mismo proceso para las estadísticas deportivas de los hitters

In [81]:
X_raw_hitter = 'Home-runs'
Y_raw = 'Sueldo'
jugador = 'Jugador'

In [82]:
# Creación de la base de datos a transformar las variables de la regresión
YX_hitter_raw = [None]*period
for year in range(0, period):
    YX_hitter_raw[year] = df_hitters_copy[year][[jugador, Y_raw, X_raw_hitter]]

# Ajuste de inflación en los salarios    
change_inflation_percentage = {2011:20.46, 2012:18.02, 2013:16.32, 2014:14.46,
                               2015:14.33, 2016:12.90, 2017:10.55, 2018:7.86,
                               2019:5.99, 2020:4.70, 2021:1}
for year in range(0, period):
    YX_hitter_raw[year][[Y_raw]] = YX_hitter_raw[year][[Y_raw]]*(1 + change_inflation_percentage[2011 + year]*0.01)
    YX_hitter_raw[year].rename(columns = {Y_raw: Y_raw + '_' + str(2011 + year)}, inplace = True)
    YX_hitter_raw[year].rename(columns = {X_raw_hitter: X_raw_hitter + '_' + str(2011 + year)}, inplace = True)

# Filtrado de base de datos
YX_hitter_panel_raw = pd.merge(YX_hitter_raw[0], YX_hitter_raw[1], how = 'outer', on = 'Jugador')

for year in range(2,period):
    YX_hitter_panel_raw = pd.merge(YX_hitter_panel_raw, YX_hitter_raw[year], how = 'outer', on = 'Jugador')

YX_hitter_panel_raw.drop_duplicates(subset = 'Jugador', inplace = True)
YX_hitter_panel_raw = YX_hitter_panel_raw.sort_values(by = 'Jugador', ascending = True)
YX_hitter_panel_raw.reset_index(drop = True, inplace = True)
YX_hitter_panel_raw.sort_index(axis = 1, inplace = True)

# Recolocación al principio de Jugador
player_column_hitter = YX_hitter_panel_raw.pop('Jugador')
YX_hitter_panel_raw.insert(0, 'Jugador', player_column_hitter)

# Creación de base de datos auxiliares  para remover los NaN
split_column_hitter = 1 + period
max_nan = 2

if Y_raw > X_raw_hitter:
    X_hitter_drop_etl = YX_hitter_panel_raw.iloc[:, 1:split_column_hitter]
else:
    X_hitter_drop_etl = YX_hitter_panel_raw.iloc[:, split_column_hitter:X_hitter_drop_etl.shape[1]]

nulls_quantity = period - max_nan
X_hitter_drop_etl = X_hitter_drop_etl.iloc[X_hitter_drop_etl[(X_hitter_drop_etl.isnull().sum(axis = 1) > nulls_quantity)].index]
X_hitter_index_drop = X_hitter_drop_etl.index

if Y_raw > X_raw_hitter:
    Y_hitter_drop_etl = YX_hitter_panel_raw.iloc[:, split_column_hitter:YX_hitter_panel_raw.shape[1]]
else:
    Y_hitter_drop_etl = YX_hitter_panel_raw.iloc[:, 1:split_column_hitter]
Y_hitter_drop_etl = Y_hitter_drop_etl.iloc[Y_hitter_drop_etl[(Y_hitter_drop_etl.isnull().sum(axis = 1) > nulls_quantity)].index]
Y_hitter_index_drop = Y_hitter_drop_etl.index

# índice de los jugadores que no tienen más de dos observaciones en el periodo de analisis
hitter_index_drop = list(set(X_hitter_index_drop).union(set(Y_hitter_index_drop)))

YX_hitter_panel_raw.drop(index = hitter_index_drop, inplace = True)
YX_hitter_panel_raw.reset_index(drop = True, inplace = True)
YX_hitter_panel_raw = YX_hitter_panel_raw.sort_values(by = 'Jugador', ascending = True)

# Borremos provicionalmente a los jugadores cuyo estadística máxima es igual a 0
if Y_raw > X_raw_hitter:
    YX_hitter_panel_raw[X_raw_hitter + '_max'] = YX_hitter_panel_raw.iloc[:, 1:split_column_hitter].max(axis = 1)
else:
    YX_hitter_panel_raw[X_raw_hitter + '_max'] = YX_hitter_panel_raw.iloc[:, split_column_hitter:YX_hitter_panel_raw.shape[1]].max(axis = 1)
YX_hitter_max_0 = YX_hitter_panel_raw[YX_hitter_panel_raw[X_raw_hitter + '_max'] == 0]
YX_hitter_max_0_index_drop = YX_hitter_max_0.index
YX_hitter_panel_raw.drop(columns = X_raw_hitter + '_max', inplace = True)
YX_hitter_panel_raw.drop(index = YX_hitter_max_0_index_drop, inplace = True)
YX_hitter_panel_raw.reset_index(drop = True, inplace = True)

# Variable X
if Y_raw > X_raw_hitter:
    X_hitter_etl = YX_hitter_panel_raw.iloc[:, 1:split_column_hitter]
else:
    X_hitter_etl = YX_hitter_panel_raw.iloc[:, split_column_hitter:YX_hitter_panel_raw.shape[1]]
hitter_max_element = X_hitter_etl.max(axis = 1)
hitter_min_element = X_hitter_etl.min(axis = 1)
hitter_max_min = (hitter_max_element + hitter_min_element)/2
dummy_hitter_names = []
X_hitter_names = []
for year in range(0,period):
    dummy_hitter_names.append("I_" + str(2011 + year))
    X_hitter_names.append("X_" + str(2011 + year))
for year in range(0,period):
    X_hitter_conditions = [
    (X_hitter_etl.iloc[:,year] == hitter_max_element) & ~(X_hitter_etl.iloc[:,year].isnull()),
    (X_hitter_etl.iloc[:,year] != hitter_max_element) & ~(X_hitter_etl.iloc[:,year].isnull())
    ]
    X_hitter_conditions_values = [0,1]
    X_hitter_etl[dummy_hitter_names[year]] = np.select(X_hitter_conditions, X_hitter_conditions_values, default = np.nan)
X_hitter_aux = X_hitter_etl.iloc[:,:period]
Dummy_hitter = X_hitter_etl.iloc[:,period:]
X_hittter_max = X_hitter_aux.max(axis = 1)
X_hitter_aux[['Max']] = np.sqrt(X_hittter_max)
for year in range(0,period):    
    X_hitter_etl['X_' + str(2011 + year)] = \
    (X_hitter_aux[X_raw_hitter + '_' + str(2011 + year)]*
    ((-1)**Dummy_hitter['I_' + str(2011 + year)]))
X_hitter_etl = X_hitter_etl.div(X_hitter_aux['Max'].values, axis = 0)
X_hitter_etl = X_hitter_etl.iloc[:,2*period:]
for i in range(0,X_hitter_etl.shape[0]):
    row = X_hitter_etl.iloc[i]
    row_length = row.size
    sustitute = []
    
    for j in range(row_length):
        if pd.isna(row[j]) != True:
            sustitute.append(row[j])
            
    sustitute_length = len(sustitute)
    
    for k in range(row_length - sustitute_length):
        sustitute.append(np.nan)
        
    for j in range(row_length):
        X_hitter_etl.iloc[i,j] = sustitute[j]

X_hitter_etl.columns = ['X_1', 'X_2', 'X_3', 'X_4', 'X_5', 'X_6', 
                         'X_7', 'X_8', 'X_9', 'X_10', 'X_11']
X_hitter_etl[['Jugador']] = YX_hitter_panel_raw[['Jugador']]
X_player_column_hitter = X_hitter_etl.pop('Jugador')
X_hitter_etl.insert(0, 'Jugador', X_player_column_hitter)

# Variable Y
if Y_raw > X_raw_hitter:
    Y_hitter_etl = YX_hitter_panel_raw.iloc[:, split_column_hitter:YX_hitter_panel_raw.shape[1]]
else:
    Y_hitter_etl = YX_hitter_panel_raw.iloc[:, 1:split_column_hitter]
for i in range(0,Y_hitter_etl.shape[0]):
    row = Y_hitter_etl.iloc[i]
    row_length = row.size
    sustitute = []
    
    for j in range(row_length):
        if pd.isna(row[j]) != True:
            sustitute.append(row[j])
            
    sustitute_length = len(sustitute)
    
    for k in range(row_length - sustitute_length):
        sustitute.append(np.nan)
        
    for j in range(row_length):
        Y_hitter_etl.iloc[i,j] = sustitute[j]

Y_hitter_etl.columns = ['w_1', 'w_2', 'w_3', 'w_4', 'w_5', 'w_6', 
                        'w_7', 'w_8', 'w_9', 'w_10', 'w_11']
Y_hitter_etl[['Jugador']] = YX_hitter_panel_raw[['Jugador']]
Y_player_column_hitter = Y_hitter_etl.pop('Jugador')
Y_hitter_etl.insert(0, 'Jugador', Y_player_column_hitter)
Y_hitter_etl_copy = Y_hitter_etl.copy()
y_etl_names = Y_hitter_etl_copy['Jugador']
for i in range(0,len(y_etl_names)):
    row = Y_hitter_etl_copy.iloc[i]
    row_length = row.size
    k = 0

    sustitute = []

    for j in range(0, row_length):
        if pd.isna(row[j]) != True:
            k = k + 1

    sustitute.append(y_etl_names[i])
    for j in range(1,k - 1):
        sustitute.append(row[j + 1]**0.5 - row[j]**0.5)

    sustitute_length = len(sustitute)

    for k in range(row_length - sustitute_length):
        sustitute.append(np.nan)

    for j in range(row_length):
        Y_hitter_etl_copy.iloc[i,j] = sustitute[j]

Y_hitter_etl_copy.columns = ['Jugador', 'Y_1', 'Y_2', 'Y_3', 'Y_4', 'Y_5', 'Y_6', 
                              'Y_7', 'Y_8', 'Y_9', 'Y_10', 'Y_11']
Y_hitter_etl = Y_hitter_etl_copy

# Base de datos
regression_hitter_list = [None]*(period-1)
for i in range(0,len(regression_hitter_list)):
    merge = pd.merge(Y_hitter_etl.iloc[:,[0,i+1]], X_hitter_etl.iloc[:,[0,i+1]])
    regression_hitter_list[i] = merge
    regression_hitter_list[i].columns = ['Jugador', 'Y', 'X']
df_article_regression_hitter = regression_hitter_list[0]
for i in range(1,len(regression_hitter_list)):
    df_article_regression_hitter = pd.concat([df_article_regression_hitter, regression_hitter_list[i]])
df_article_regression_hitter = df_article_regression_hitter.sort_values(by = 'Jugador', ascending = True)
df_article_regression_hitter.reset_index(drop = True, inplace = True)
df_article_regression_hitter.dropna(inplace = True)
df_article_regression_hitter.reset_index(drop = True, inplace = True)

In [83]:
df_article_regression_hitter

,Jugador,Y,X
0,A.J. Burnett,-471.040165,-0.0
1,A.J. Pierzynski,1218.568225,-1.212678
2,A.J. Pierzynski,199.175332,-2.182821
3,A.J. Pierzynski,-2515.731853,4.123106
4,A.J. Pollock,300.847677,-2.630384
...,...,...,...
356,Zack Greinke,-8.745908,-0.0
357,Zack Greinke,-64.819869,-0.0
358,Zack Greinke,849.383256,-1.154701
359,Zack Greinke,-108.975642,-0.57735


Vemos que el experimento fue un éxito, por lo que ya podemos pasar a una función este algoritmo.

## Función de ETL

Puesto que ya funcionó el algoritmo, ahora podemos pasarlo a una función que se pueda utilizar para cualquier tipo de base de datos. Por otro lado, haremos una modificación para que regrese la base de datos generada para la regresión y a la vez que también regrese al modelo con los estimadores

In [84]:
def etl_regression(X_raw, Y_raw, df_copy):
    # Creación de la base de datos a transformar las variables de la regresión
    jugador = 'Jugador'
    YX_raw = [None]*period
    
    for year in range(0, period):
        YX_raw[year] = df_copy[year][[jugador, Y_raw, X_raw]]

    # Ajuste de inflación en los salarios    
    change_inflation_percentage = {2011:20.46, 2012:18.02, 2013:16.32, 2014:14.46,
                                   2015:14.33, 2016:12.90, 2017:10.55, 2018:7.86,
                                   2019:5.99, 2020:4.70, 2021:1}
    for year in range(0, period):
        YX_raw[year][[Y_raw]] = YX_raw[year][[Y_raw]]*(1 + change_inflation_percentage[2011 + year]*0.01)
        YX_raw[year].rename(columns = {Y_raw: Y_raw + '_' + str(2011 + year)}, inplace = True)
        YX_raw[year].rename(columns = {X_raw: X_raw + '_' + str(2011 + year)}, inplace = True)

    # Filtrado de base de datos
    YX_panel_raw = pd.merge(YX_raw[0], YX_raw[1], how = 'outer', on = 'Jugador')

    for year in range(2,period):
        YX_panel_raw = pd.merge(YX_panel_raw, YX_raw[year], how = 'outer', on = 'Jugador')

    YX_panel_raw.drop_duplicates(subset = 'Jugador', inplace = True)
    YX_panel_raw = YX_panel_raw.sort_values(by = 'Jugador', ascending = True)
    YX_panel_raw.reset_index(drop = True, inplace = True)
    YX_panel_raw.sort_index(axis = 1, inplace = True)

    # Recolocación al principio de Jugador
    player_column = YX_panel_raw.pop('Jugador')
    YX_panel_raw.insert(0, 'Jugador', player_column)

    # Creación de base de datos auxiliares  para remover los NaN
    split_column = 1 + period
    max_nan = 2

    if Y_raw > X_raw:
        X_drop_etl = YX_panel_raw.iloc[:, 1:split_column]
    else:
        X_drop_etl = YX_panel_raw.iloc[:, split_column:YX_panel_raw.shape[1]]

    nulls_quantity = period - max_nan
    X_drop_etl = X_drop_etl.iloc[X_drop_etl[(X_drop_etl.isnull().sum(axis = 1) > nulls_quantity)].index]
    X_index_drop = X_drop_etl.index

    if Y_raw > X_raw:
        Y_drop_etl = YX_panel_raw.iloc[:, split_column:YX_panel_raw.shape[1]]
    else:
        Y_drop_etl = YX_panel_raw.iloc[:, 1:split_column]
    Y_drop_etl = Y_drop_etl.iloc[Y_drop_etl[(Y_drop_etl.isnull().sum(axis = 1) > nulls_quantity)].index]
    Y_index_drop = Y_drop_etl.index

    # índice de los jugadores que no tienen más de dos observaciones en el periodo de analisis
    index_drop = list(set(X_index_drop).union(set(Y_index_drop)))

    YX_panel_raw.drop(index = index_drop, inplace = True)
    YX_panel_raw.reset_index(drop = True, inplace = True)
    YX_panel_raw = YX_panel_raw.sort_values(by = 'Jugador', ascending = True)

    # Borremos provicionalmente a los jugadores cuyo estadística máxima es igual a 0
    if Y_raw > X_raw:
        YX_panel_raw[X_raw + '_max'] = YX_panel_raw.iloc[:, 1:split_column].max(axis = 1)
    else:
        YX_panel_raw[X_raw + '_max'] = YX_panel_raw.iloc[:, split_column:YX_panel_raw.shape[1]].max(axis = 1)
    YX_max_0 = YX_panel_raw[YX_panel_raw[X_raw + '_max'] == 0]
    YX_max_0_index_drop = YX_max_0.index
    YX_panel_raw.drop(columns = X_raw + '_max', inplace = True)
    YX_panel_raw.drop(index = YX_max_0_index_drop, inplace = True)
    YX_panel_raw.reset_index(drop = True, inplace = True)

    # Variable X
    if Y_raw > X_raw:
        X_etl = YX_panel_raw.iloc[:, 1:split_column]
    else:
        X_etl = YX_panel_raw.iloc[:, split_column:YX_panel_raw.shape[1]]
    max_element = X_etl.max(axis = 1)
    min_element = X_etl.min(axis = 1)
    max_min = (max_element + min_element)/2
    dummy_names = []
    X_names = []
    for year in range(0,period):
        dummy_names.append("I_" + str(2011 + year))
        X_names.append("X_" + str(2011 + year))
    for year in range(0,period):
        X_conditions = [
        (X_etl.iloc[:,year] == max_element) & ~(X_etl.iloc[:,year].isnull()),
        (X_etl.iloc[:,year] != max_element) & ~(X_etl.iloc[:,year].isnull())
        ]
        X_conditions_values = [0,1]
        X_etl[dummy_names[year]] = np.select(X_conditions, X_conditions_values, default = np.nan)
    X_aux = X_etl.iloc[:,:period]
    Dummy = X_etl.iloc[:,period:]
    X_max = X_aux.max(axis = 1)
    X_aux[['Max']] = np.sqrt(X_max)
    for year in range(0,period):    
        X_etl['X_' + str(2011 + year)] = \
        (X_aux[X_raw + '_' + str(2011 + year)]*
        ((-1)**Dummy['I_' + str(2011 + year)]))
    X_etl = X_etl.div(X_aux['Max'].values, axis = 0)
    X_etl = X_etl.iloc[:,2*period:]
    for i in range(0,X_etl.shape[0]):
        row = X_etl.iloc[i]
        row_length = row.size
        sustitute = []

        for j in range(row_length):
            if pd.isna(row[j]) != True:
                sustitute.append(row[j])

        sustitute_length = len(sustitute)

        for k in range(row_length - sustitute_length):
            sustitute.append(np.nan)

        for j in range(row_length):
            X_etl.iloc[i,j] = sustitute[j]

    X_etl.columns = ['X_1', 'X_2', 'X_3', 'X_4', 'X_5', 'X_6', 
                     'X_7', 'X_8', 'X_9', 'X_10', 'X_11']
    X_etl[['Jugador']] = YX_panel_raw[['Jugador']]
    X_player_column = X_etl.pop('Jugador')
    X_etl.insert(0, 'Jugador', X_player_column)

    # Variable Y
    if Y_raw > X_raw:
        Y_etl = YX_panel_raw.iloc[:, split_column:YX_panel_raw.shape[1]]
    else:
        Y_etl = YX_panel_raw.iloc[:, 1:split_column]
    for i in range(0,Y_etl.shape[0]):
        row = Y_etl.iloc[i]
        row_length = row.size
        sustitute = []

        for j in range(row_length):
            if pd.isna(row[j]) != True:
                sustitute.append(row[j])

        sustitute_length = len(sustitute)

        for k in range(row_length - sustitute_length):
            sustitute.append(np.nan)

        for j in range(row_length):
            Y_etl.iloc[i,j] = sustitute[j]

    Y_etl.columns = ['w_1', 'w_2', 'w_3', 'w_4', 'w_5', 'w_6', 
                     'w_7', 'w_8', 'w_9', 'w_10', 'w_11']
    Y_etl[['Jugador']] = YX_panel_raw[['Jugador']]
    Y_player_column = Y_etl.pop('Jugador')
    Y_etl.insert(0, 'Jugador', Y_player_column)
    Y_etl_copy = Y_etl.copy()
    y_etl_names = Y_etl_copy['Jugador']
    for i in range(0,len(y_etl_names)):
        row = Y_etl_copy.iloc[i]
        row_length = row.size
        k = 0

        sustitute = []

        for j in range(0, row_length):
            if pd.isna(row[j]) != True:
                k = k + 1

        sustitute.append(y_etl_names[i])
        for j in range(1,k - 1):
            sustitute.append(row[j + 1]**0.5 - row[j]**0.5)

        sustitute_length = len(sustitute)

        for k in range(row_length - sustitute_length):
            sustitute.append(np.nan)

        for j in range(row_length):
            Y_etl_copy.iloc[i,j] = sustitute[j]

    Y_etl_copy.columns = ['Jugador', 'Y_1', 'Y_2', 'Y_3', 'Y_4', 'Y_5', 'Y_6', 
                          'Y_7', 'Y_8', 'Y_9', 'Y_10', 'Y_11']
    Y_etl = Y_etl_copy

    # Base de datos
    regression_list = [None]*(period-1)
    for i in range(0,len(regression_list)):
        merge = pd.merge(Y_etl.iloc[:,[0,i+1]], X_etl.iloc[:,[0,i+1]])
        regression_list[i] = merge
        regression_list[i].columns = ['Jugador', 'Y', 'X']
    df_article_regression = regression_list[0]
    for i in range(1,len(regression_list)):
        df_article_regression = pd.concat([df_article_regression, regression_list[i]])
    df_article_regression = df_article_regression.sort_values(by = 'Jugador', ascending = True)
    df_article_regression.reset_index(drop = True, inplace = True)
    df_article_regression.dropna(inplace = True)
    df_article_regression.reset_index(drop = True, inplace = True)
    
    # Construimos la regresion
    Y = df_article_regression['Y'].tolist()
    X = df_article_regression['X'].tolist()
    X = sm.add_constant(X)
    model = sm.OLS(Y, X).fit()
    
    # Regresamos como argumento al modelo
    return [df_article_regression, model]

In [85]:
X_raw_pitcher = 'Inning_pitched'
Y_raw = 'Sueldo'
df_copy = df_pitchers_copy

In [86]:
etl_regression(X_raw_pitcher, Y_raw, df_copy)

[           Jugador           Y          X
 0     A.J. Burnett -471.040165  14.618481
 1     Aaron Harang  647.699528  13.405223
 2     Aaron Harang -138.427417 -10.689863
 3    Adam Ottavino -368.340054  -2.042177
 4    Adam Ottavino  -18.852808  -7.398706
 ..             ...         ...        ...
 420   Zack Greinke    6.483182 -13.528012
 421   Zack Greinke  -75.049658 -13.113161
 422   Zack Greinke  -64.819869 -10.286993
 423   Zack Greinke  740.407614 -13.113161
 424   Zack Greinke  754.087221  -11.51858
 
 [425 rows x 3 columns],